# **Part 5 - Altres Arquitectures**

De moment hem treballat únicament amb xarxes de tipus Resnet, com ``resnet18`` o ``resnet152``. No obstant, pytorch (i, per extensió, ``fastai``) ofereix moltes altres xarxes i arquitectures prefabricades per fer transfer learning. En aquest *Notebook* veurem algunes d'elles: explicarem breument la seva història i context, estudiarem la seva arquitectura i compararem els seus resultats classificant les imatges del Dataset d'aquesta pràctica.

### **Carregar llibreries**

Carreguem les llibreries i mòduls habituals que hem fet servir en els altres exemples de la pràctica:

In [11]:
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
from fastbook import *
fastbook.setup_book()
from fastai.callback.fp16 import *
import torch

### **Carregar Dataset (només Colab)**

Si estem executant aquest Notebook des de Colab, necessitarem tenir el dataset guardat al nostre Drive en format .zip i executar la següent cel·la de codi per carregar-lo a aquesta sessió:

In [12]:
!unzip /content/gdrive/MyDrive/dataset_CIFAR10.zip
path = "/content/"

Archive:  /content/gdrive/MyDrive/dataset_CIFAR10.zip
replace train/airplane/0000.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


Altrament, **suposarem que el dataset es troba descomprimit en la mateixa carpeta des d'on estem executant aquest Notebook**.

### **DataLoader**

Per tots els exemples d'aquest Notebook farem servir el següent DataLoader:

In [ ]:
dls = ImageDataLoaders.from_folder(
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    path = path,
    train='train',
    valid='validation',
    seed=42,
    batch_tfms=aug_transforms(size=64),
    bs = 64)

Noti's que hem ampliat la mida de les imatges de 32x32 a 64x64. La raó es que en moltes de les arquitectures que veurem, la dimensió de la imatge es redueix molt al propagar-se cap endavant; si usem una mida massa petita en les imatges d'input la dimensió arriba a zero en alguns casos (per exemple amb Alexnet), i ``fastai`` llança un error perquè no pot continuar propagant la imatge. <br>

També hem reduït la mida del batch size a 64, perquè ens hem trobat amb alguns problemes de memòria si executavem algunes d'aquestes arquitectures a la plataforma Colab, malgrat que no hauria d'haver-hi problema si s'està executant aquest Notebook des d'un ordinador fixe de prestacions mitjanes (pels estàndars actuals).



---




###**1. Alexnet**

Alexnet és una xarxa neuronal convolucional de 8 capes de profunditat entrenada per classificar imatges de 227x227 píxels del dataset Imagenet. Va ser la guanyadora del *ImageNet Large Scale Visual Recognition Competition* (*ILSVRC*) de l'any 2012, aconseguint una precisió propera al 85%. Malgrat que aquesta puntuació a dia d'avui no seria sorprenent, en aquell moment fóu una fita molt important.

<table style="text-align: center;">
<tr>
  <td>
   <img src="https://neurohive.io/wp-content/uploads/2018/10/AlexNet-1.png"/>
  </td>
</tr>
<tr>
  <th>Topologia de la xarxa Alexnet original. Imatge extreta de <a href = https://neurohive.io/en/popular-networks/alexnet-imagenet-classification-with-deep-convolutional-neural-networks/>Neurohive</a>.</th>
</tr>
</table>

Anem, doncs, a crear un objecte de tipus ``vision_learner`` que utilitzi l'arquitectura Alexnet:

In [14]:
alex_learner = vision_learner(dls, alexnet, metrics=[accuracy, error_rate]).to_fp16()

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


L'entrenem durant 12 èpoques, congelant les dues primeres:

In [5]:
alex_learner.fine_tune(10, freeze_epochs=2)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:02<00:00, 114MB/s]


epoch,train_loss,valid_loss,accuracy,error_rate,time
0,1.505041,1.090612,0.615800,0.384200,01:03
1,1.262683,1.062366,0.619900,0.380100,00:55


epoch,train_loss,valid_loss,accuracy,error_rate,time
0,0.972461,0.805696,0.714000,0.286000,01:04
1,0.893869,0.727208,0.741500,0.258500,01:07
2,0.802876,0.683696,0.760600,0.239400,01:07
3,0.735399,0.643175,0.770100,0.229900,01:01
4,0.653814,0.611275,0.781500,0.218500,01:02
5,0.614006,0.588301,0.794000,0.206000,01:03
6,0.546973,0.560392,0.803100,0.196900,01:03
7,0.520514,0.548012,0.806000,0.194000,01:00
8,0.488031,0.545835,0.807700,0.192300,00:59
9,0.519344,0.543788,0.809000,0.191000,01:01


Veiem que hem aconseguit un 80% de precisió re-entrenant la xarxa amb el nostre dataset, que no està molt allunyat dels resultats que va obtenir aquesta xarxa en el seu moment.

<br>

### **2. Squeezenet**

[En paraules dels seus creadors](https://arxiv.org/abs/1602.07360), Squeezenet fóu un intent de construir una xarxa capaç d'aconseguir la mateixa precisió que Alexnet, però fent servir 50 vegades menys paràmetres i menys de 0.5MB de memòria per guardar el model. La que provarem avui serà la versió 1.1, que està lleugerament més refinada que la versió original.




<table style="text-align: center;">
<tr>
  <td>
   <img src="https://www.researchgate.net/publication/335357358/figure/fig4/AS:795145175842817@1566588791117/The-architecture-of-SqueezeNet-11.ppm"/>
  </td>
</tr>
<tr>
  <th>Topologia de la xarxa Squeezenet 1.1. Imatge extreta de <a href = https://www.researchgate.net/figure/The-architecture-of-SqueezeNet-11_fig4_335357358>WANGHU & RUOXUE (2019), "Real-Time Driver-Drowsiness Detection System Using Facial Features"</a>.</th>
</tr>
</table>

Anem a comprovar si aquesta arquitectura realment està a l'altura de tals afirmacions...

Per a aquesta xarxa ens serveix el mateix DataLoader que hem fet servir per a la xarxa Alexnet, així que podem passar a crear directament la xarxa:

In [7]:
squish_learner = vision_learner(dls, squeezenet1_1, metrics=[accuracy, error_rate]).to_fp16()

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/squeezenet1_1-b8a52dc0.pth" to /root/.cache/torch/hub/checkpoints/squeezenet1_1-b8a52dc0.pth
100%|██████████| 4.73M/4.73M [00:00<00:00, 27.1MB/s]


Una vegada creada la xarxa, procedim a entrenar-la. Idealment, hauriem d'obtenir uns resultats similars als que ens ha donat Alexnet, però amb molt menys temps d'entrenament.

In [8]:
squish_learner.fine_tune(10, freeze_epochs=2)

epoch,train_loss,valid_loss,accuracy,error_rate,time
0,1.425406,1.087537,0.623100,0.376900,01:06
1,1.195729,1.015506,0.631800,0.368200,01:01


epoch,train_loss,valid_loss,accuracy,error_rate,time
0,0.913902,0.757032,0.736300,0.263700,01:12
1,0.823293,0.679351,0.759600,0.240400,01:09
2,0.795195,0.679042,0.759200,0.240800,01:10
3,0.702959,0.590671,0.790800,0.209200,01:15
4,0.659900,0.580694,0.795100,0.204900,01:10
5,0.620579,0.545290,0.806100,0.193900,01:15
6,0.596448,0.510637,0.820400,0.179600,01:09
7,0.531956,0.499747,0.820700,0.179300,01:13
8,0.526310,0.489126,0.825800,0.174200,01:09
9,0.509471,0.487938,0.827000,0.173000,01:10


Curiosament, hem acabat obtenint prop d'un 83% de precisió, que és una mica més que l'exemple d'Alexnet.

<br>

### **3. VGG16 i VGG19**

La VGG-16 fóu l'Arquitectura guanyadora de l'*ILSVRC* de l'any 2014. Va suposar un salt qualitatiu envers les arquitectures líders del moment com Alexnet, aconseguint un impresionant 92.7% de precisió. Com el seu nom indica, es tracta d'una arquitectura de 16 capes convolucionals:

<table style="text-align: center;">
<tr>
  <td>
   <img src="https://miro.medium.com/v2/resize:fit:720/0*0M8CobXpNwFDCmOQ"/>
  </td>
</tr>
<tr>
  <th>Topologia de la xarxa VGG-16. Imatge extreta de <a https://medium.com/@mygreatlearning/everything-you-need-to-know-about-vgg16-7315defb5918>Medium</a>.</th>
</tr>
</table>

In [15]:
vgg16_learner = vision_learner(dls, vgg16_bn, metrics=[accuracy, error_rate]).to_fp16()

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth
100%|██████████| 528M/528M [00:07<00:00, 78.3MB/s]


In [16]:
vgg16_learner.fine_tune(10, freeze_epochs=2)

epoch,train_loss,valid_loss,accuracy,error_rate,time
0,1.441903,1.010889,0.650700,0.349300,01:12
1,0.970461,0.752161,0.737600,0.262400,01:09


epoch,train_loss,valid_loss,accuracy,error_rate,time
0,0.598352,0.452743,0.845700,0.154300,01:21
1,0.466979,0.368632,0.873600,0.126400,01:17
2,0.408214,0.325256,0.887200,0.112800,01:13
3,0.324227,0.314983,0.893900,0.106100,01:17
4,0.249876,0.304633,0.896600,0.103400,01:16
5,0.187534,0.298017,0.905300,0.094700,01:14
6,0.135919,0.302177,0.905200,0.094800,01:17
7,0.104642,0.286803,0.912100,0.087900,01:16
8,0.076573,0.281018,0.914100,0.085900,01:14
9,0.060850,0.278888,0.914600,0.085400,01:15


Hem aconseguit un 91.4% de precisió, que està molt a prop del màxim esperable per a aquest tipus de xarxa. <br>

Provem ara la VGG-19. Essencialment és el mateix tipus de xarxa, pero apilant 19 capes convolucionals en comptes de 16.

In [17]:
vgg19_learner = vision_learner(dls, vgg19_bn, metrics=[accuracy, error_rate]).to_fp16()

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to /root/.cache/torch/hub/checkpoints/vgg19_bn-c79401a0.pth
100%|██████████| 548M/548M [00:15<00:00, 37.4MB/s]


In [18]:
vgg19_learner.fine_tune(10, freeze_epochs=2)

epoch,train_loss,valid_loss,accuracy,error_rate,time
0,1.518708,1.066034,0.639900,0.360100,01:12
1,1.006780,0.786184,0.722700,0.277300,01:16


epoch,train_loss,valid_loss,accuracy,error_rate,time
0,0.593567,0.435894,0.847700,0.152300,01:23
1,0.452955,0.354488,0.876000,0.124000,01:24
2,0.395166,0.334185,0.885000,0.115000,01:22
3,0.324196,0.296501,0.898900,0.101100,01:21
4,0.250396,0.295714,0.900100,0.099900,01:20
5,0.190576,0.288921,0.904600,0.095400,01:20
6,0.139383,0.266869,0.915800,0.084200,01:20
7,0.087548,0.270891,0.919200,0.080800,01:20
8,0.075082,0.268692,0.922000,0.078000,01:20
9,0.055379,0.272057,0.921000,0.079000,01:19


Aquesta vegada hem obtingut 92.1%, millorant lleugerament el resultat de la VGG-16.